In [1]:
from nn import *
from metrics import accuracy
from tools import normalize

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

In [4]:
X_train, y_train = train_df.iloc[:, 1:], train_df.iloc[:, 0]
X_test, y_test = test_df.iloc[:, 1:], test_df.iloc[:, 0]

In [5]:
X_test = (X_test - X_train.mean()) / X_train.std()
X_train = (X_train - X_train.mean()) / X_train.std()

In [6]:
y_map = {"B": 0, "M":1}
y_train = y_train.map(y_map).values[:, np.newaxis].astype(np.float64)
y_test = y_test.map(y_map).values[:, np.newaxis].astype(np.float64)

In [7]:
def build_model(initializer='normal'):
    nn = NeuralNetwork(loss=CrossEntropy(), optimizer='gd', verbose=False)
    nn.add(Dense(64, activation=Relu(), initializer=initializer))
    nn.add(Dense(1, activation=Sigmoid(), initializer=initializer))
    return nn

In [8]:
n_episodes = 5
inits = ['normal', 'xavier', 'he', 'he_unif']
for init in inits:
    scores = []
    np.random.seed(42)
    for ep in range(n_episodes):
        nn = build_model(initializer=init)
        nn.fit(X_train.values, y_train, n_epochs=5, batch_size=32)
        acc_train = accuracy(y_train, nn.predict(X_train) > .5)
        acc_test = accuracy(y_test, nn.predict(X_test) > .5)
        scores.append([acc_train, acc_test])
    scores = np.array(scores)
    mean_train_acc, mean_test_acc = scores.mean(axis=0)
    print(f"init={init} | mean_train_acc={mean_train_acc} | mean_test_acc={mean_test_acc}")

init=normal | mean_train_acc=0.6455399061032864 | mean_test_acc=0.5748251748251747
init=xavier | mean_train_acc=0.7943661971830986 | mean_test_acc=0.7412587412587412
init=he | mean_train_acc=0.9253521126760564 | mean_test_acc=0.8965034965034965
init=he_unif | mean_train_acc=0.94037558685446 | mean_test_acc=0.923076923076923
